* issue [#296](https://github.com/salgo60/Stockholm_Archipelago_Trail/issues/296) 
* this Notebook [296_did_you_know.ipynb](296_did_you_know.ipynb)

In [1]:
import time
import datetime  
start_time = time.time()
start_str = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")
print(f"Started: {start_str}")


Started: 2025-11-01 12:28


In [2]:
import openai
openai.api_key = os.getenv("OPENAI_API_KEY")


NameError: name 'os' is not defined

In [ ]:
def fetch_sat_connected_items(limit=1000):
    query = f"""
    SELECT ?item ?itemLabel ?itemDescription ?coord ?_image ?OSM ?objectLabel
           ?articleEN ?articleSV
           ?Facebook ?Instagram ?InstagramL ?Googlemap
    WHERE {{
      ?item wdt:P6104 wd:Q134294510;
            wdt:P625 ?coord.

      OPTIONAL {{ ?item wdt:P10689 ?OSMway. }}
      OPTIONAL {{ ?item wdt:P402 ?OSMrel. }}
      OPTIONAL {{ ?item wdt:P11693 ?OSMnode. }}
      OPTIONAL {{ ?item wdt:P31 ?object. }}

      OPTIONAL {{ ?item wdt:P2003 ?InstaU. }}
      OPTIONAL {{ ?item wdt:P4173 ?InstaL. }}
      OPTIONAL {{ ?item wdt:P2013 ?FB. }}
      OPTIONAL {{ ?item wdt:P3749 ?Gmap. }}

      BIND(URI(CONCAT("https://www.facebook.com/", ?FB)) AS ?Facebook)
      BIND(URI(CONCAT("https://www.instagram.com/", ?InstaU)) AS ?Instagram)
      BIND(URI(CONCAT("https://www.instagram.com/explore/locations/", ?InstaL)) AS ?InstagramL)
      BIND(URI(CONCAT("https://www.google.com/maps?cid=", ?Gmap)) AS ?Googlemap)

      OPTIONAL {{
        ?articleEN schema:about ?item ;
                   schema:inLanguage "en" ;
                   schema:isPartOf [ wikibase:wikiGroup "wikipedia" ] .
      }}
      OPTIONAL {{
        ?articleSV schema:about ?item ;
                   schema:inLanguage "sv" ;
                   schema:isPartOf [ wikibase:wikiGroup "wikipedia" ] .
      }}
      # Official website
      OPTIONAL {{ ?item wdt:P856 ?Website. }}

      OPTIONAL {{ ?item wdt:P18 ?_image. }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "sv,en". }}
    }}
    LIMIT {limit}
    """
    sparql = SPARQLWrapper(ENDPOINT)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()["results"]["bindings"]




In [ ]:
def fetch_sat_connected_items(limit=400):
    query = f"""
    SELECT ?item ?itemLabel ?itemDescription ?coord ?_image ?OSM ?objectLabel
           ?articleEN ?articleSV
           ?Facebook ?Instagram ?InstagramL ?Googlemap
    WHERE {{
      ?item wdt:P6104 wd:Q134294510;
            wdt:P625 ?coord.

      OPTIONAL {{ ?item wdt:P10689 ?OSMway. }}
      OPTIONAL {{ ?item wdt:P402 ?OSMrel. }}
      OPTIONAL {{ ?item wdt:P11693 ?OSMnode. }}
      OPTIONAL {{ ?item wdt:P31 ?object. }}

      OPTIONAL {{ ?item wdt:P2003 ?InstaU. }}
      OPTIONAL {{ ?item wdt:P4173 ?InstaL. }}
      OPTIONAL {{ ?item wdt:P2013 ?FB. }}
      OPTIONAL {{ ?item wdt:P3749 ?Gmap. }}

      # Official website
      OPTIONAL {{ ?item wdt:P856 ?Website. }}


      BIND(URI(CONCAT("https://www.facebook.com/", ?FB)) AS ?Facebook)
      BIND(URI(CONCAT("https://www.instagram.com/", ?InstaU)) AS ?Instagram)
      BIND(URI(CONCAT("https://www.instagram.com/explore/locations/", ?InstaL)) AS ?InstagramL)
      BIND(URI(CONCAT("https://www.google.com/maps?cid=", ?Gmap)) AS ?Googlemap)

      OPTIONAL {{
        ?articleEN schema:about ?item ;
                   schema:inLanguage "en" ;
                   schema:isPartOf [ wikibase:wikiGroup "wikipedia" ] .
      }}
      OPTIONAL {{
        ?articleSV schema:about ?item ;
                   schema:inLanguage "sv" ;
                   schema:isPartOf [ wikibase:wikiGroup "wikipedia" ] .
      }}

      OPTIONAL {{ ?item wdt:P18 ?_image. }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "sv,en". }}
    }}
    LIMIT {limit}
    """
    sparql = SPARQLWrapper(ENDPOINT)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()["results"]["bindings"]

import html

def generate_fact(item, lang="sv"):
    """Generate a compact popup with icons for links and social media"""
    label = item.get("itemLabel", {}).get("value", "")
    desc = item.get("itemDescription", {}).get("value", "")
    img = item.get("_image", {}).get("value", "")
    wd_uri = item.get("item", {}).get("value", "")

    label_safe = html.escape(label)
    desc_safe = html.escape(desc)

    # Intro text
    if lang == "sv":
        fact = f"Visste du att <b>{label_safe}</b>? {desc_safe or ''}"
    else:
        fact = f"Did you know that <b>{label_safe}</b>? {desc_safe or ''}"

    # --- ICON LINKS SECTION ---
    links_html = ""

    # icon style
    icon_style = (
        "display:inline-block; width:28px; height:28px; line-height:28px; "
        "text-align:center; border-radius:50%; background:#f0f0f0; "
        "margin:2px; font-size:16px; text-decoration:none;"
    )

    def make_icon(url, emoji, title):
        return (
            f"<a href='{url}' target='_blank' title='{title}' style='{icon_style}'>{emoji}</a>"
        )

    # Wikidata
    if wd_uri:
        links_html += make_icon(wd_uri, "🔗", "Wikidata")

    # Wikipedia (Swedish / English)
    if "articleSV" in item and "value" in item["articleSV"]:
        links_html += make_icon(item["articleSV"]["value"], "🇸🇪", "Wikipedia (sv)")
    if "articleEN" in item and "value" in item["articleEN"]:
        links_html += make_icon(item["articleEN"]["value"], "🇬🇧", "Wikipedia (en)")

    # Official website
    if "Website" in item and "value" in item["Website"]:
        links_html += make_icon(item["Website"]["value"], "🌐", "Official website")

    # Social media + maps
    emoji_map = {
        "Facebook": ("📘", "Facebook"),
        "Instagram": ("📸", "Instagram"),
        "InstagramL": ("📍", "Instagram location"),
        "Googlemap": ("🗺️", "Google Maps"),
    }
    for key, (emoji, title) in emoji_map.items():
        if key in item and "value" in item[key]:
            links_html += make_icon(item[key]["value"], emoji, title)

    # Combine
    if links_html:
        fact += f"<div style='margin-top:6px;'>{links_html}</div>"

    # Add image
    if img:
        fact += f"<br><img src='{html.escape(img)}' width='250px' style='margin-top:8px;border-radius:10px;'>"

    return fact


In [ ]:
import openai
import os

# Make sure your key is set:
# os.environ["OPENAI_API_KEY"] = "sk-..."

def polish_fact_with_ai(fact_text, lang="en"):
    """Use GPT to rewrite the fact naturally"""
    system_prompt = (
        "You are a helpful hiking guide. "
        "Rewrite the following text as a short, friendly 'Did you know?' message for a hiking map. "
        "Keep it factual and concise."
    )
    if lang == "sv":
        system_prompt = (
            "Du är en hjälpsam guide för vandrare. "
            "Skriv om texten som en kort, trevlig 'Visste du att...'-mening på svenska. "
            "Behåll fakta, gör texten naturlig och flytande."
        )

    try:
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": fact_text},
            ],
            temperature=0.7,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print("AI polish failed:", e)
        return fact_text


In [ ]:
import json
import folium

def add_trail_layer(m, trail_file="SAT_full.geojson"):
    """Add the main SAT trail GeoJSON line to the map."""
    try:
        with open(trail_file, "r", encoding="utf-8") as f:
            trail_data = json.load(f)

        folium.GeoJson(
            trail_data,
            name="SAT Trail",
            style_function=lambda feature: {
                "color": "#ff6600",  # warm orange
                "weight": 4,
                "opacity": 0.8
            },
            highlight_function=lambda feature: {"weight": 6, "color": "#ff3300"},
            tooltip=folium.Tooltip("🏃 Stockholm Archipelago Trail"),
        ).add_to(m)

        print(f"✅ Trail '{trail_file}' added to map.")
    except FileNotFoundError:
        print(f"⚠️ Trail file '{trail_file}' not found — skipping.")
    return m


In [ ]:
from SPARQLWrapper import SPARQLWrapper, JSON
import folium
import html
import random

ENDPOINT = "https://query.wikidata.org/sparql"
OUTPUT_FILE = "295_Did_You_Know.html"
FACT_LIMIT = 1000  # you can raise or lower as needed


def fetch_sat_connected_items(limit=FACT_LIMIT):
    """Fetch items connected to the Stockholm Archipelago Trail via P6104"""
    query = f"""
    SELECT ?item ?itemLabel ?itemDescription ?coord ?_image ?OSM ?objectLabel ?article WHERE {{
      ?item wdt:P6104 wd:Q134294510;  # linked to Stockholm Archipelago Trail project
            wdt:P625 ?coord.
      OPTIONAL {{ ?item wdt:P10689 ?OSMway. }}
      OPTIONAL {{ ?item wdt:P402 ?OSMrel. }}
      OPTIONAL {{ ?item wdt:P11693 ?OSMnode. }}
      OPTIONAL {{ ?item wdt:P31 ?object. }}
      BIND(URI(CONCAT("https://www.openstreetmap.org/node/", ?OSMnode)) AS ?OSM)
      BIND(URI(CONCAT("https://www.openstreetmap.org/way/", ?OSMway)) AS ?OSM)
      BIND(URI(CONCAT("https://www.openstreetmap.org/relation/", ?OSMrel)) AS ?OSM)
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "sv,en". }}
      OPTIONAL {{
        ?article schema:about ?item ;
                 schema:inLanguage "en" ;
                 schema:isPartOf [ wikibase:wikiGroup "wikipedia" ] .
      }}
      OPTIONAL {{ ?item wdt:P18 ?_image. }}
    }}
    LIMIT {limit}
    """
    #print(query)
    sparql = SPARQLWrapper(ENDPOINT)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()["results"]["bindings"]


def get_coords(item):
    coord = item.get("coord", {}).get("value", "")
    if coord.startswith("Point("):
        lon, lat = coord[6:-1].split(" ")
        return float(lat), float(lon)
    return None, None

from folium import IFrame, Popup

def build_fact_map(items, lang="sv"):
    m = folium.Map(location=[59.3, 18.8], zoom_start=8, tiles="CartoDB positron")

    for item in items:
        lat, lon = get_coords(item)
        if not lat or not lon:
            continue

        # Generate the popup HTML
        popup_html = generate_fact(item, lang)

        # Wrap inside an IFrame (prevents HTML sanitization)
        iframe = IFrame(popup_html, width=320, height=300)
        popup = Popup(iframe, max_width=320)

        folium.Marker(
            [lat, lon],
            popup=popup,
            tooltip=item["itemLabel"]["value"],
            icon=folium.Icon(color="blue", icon="info-sign"),
        ).add_to(m)

    return m



if __name__ == "__main__":
    print("Fetching SAT-connected items via P6104...")
    items = fetch_sat_connected_items()
    print(f"Fetched {len(items)} items.")

    m = build_fact_map(items, lang="sv")
    add_trail_layer(m)
    m.save(OUTPUT_FILE)
    print(f"✅ Map saved as {OUTPUT_FILE}")


In [ ]:
m


In [ ]:
item = items[0]
print(generate_fact(item, lang="sv"))